<a href="https://colab.research.google.com/github/bghaendler/BJBS-AI-Lab/blob/master/Bond_Analytics_Functions_Yield_to_Maturity_DV01_Credit_Spread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bond Analytics Functions

This code provides three functions to calculate various analytics measures for a bond:

1. `calculate_ytm(price, coupon, face_value, time_to_maturity, frequency)`: Calculates the Yield to Maturity (YTM) for a bond.
2. `calculate_dv01(price, coupon, face_value, time_to_maturity, frequency)`: Calculates the DV01 (dollar value of a basis point) for a bond.
3. `calculate_credit_spread(price, coupon, face_value, time_to_maturity, frequency, risk_free_rate)`: Calculates the Credit Spread for a bond.

### Input Parameters:

- `price`: The current price of the bond.
- `coupon`: The coupon payment received per period.
- `face_value`: The face value or par value of the bond.
- `time_to_maturity`: The time to maturity of the bond in years.
- `frequency`: The number of coupon payments per year.
- `risk_free_rate`: The risk-free interest rate used to calculate the credit spread.

### Calculating Yield to Maturity (YTM):

The YTM represents the internal rate of return (IRR) of a bond, which equates the present value of all future cash flows (coupon payments and face value) to the current market price. The YTM is calculated iteratively using the Newton-Raphson method.

The formula to calculate the YTM for a bond is:

$$
\text{Present Value} = \sum_{t=1}^{n} \frac{{C / F}}{{(1 + r / F)^t}} + \frac{{F}}{{(1 + r / F)^n}}
$$

where:
- $C$ is the coupon payment per period.
- $F$ is the face value of the bond.
- $r$ is the yield to maturity (YTM).
- $n$ is the total number of periods until maturity.
- $t$ is the current period.

The function `calculate_ytm` uses this formula to calculate the YTM for a given bond.

### Calculating DV01:

The DV01 (dollar value of a basis point) measures the change in the bond price for a 0.01% (1 basis point) change in yield to maturity. It is an important measure for bond risk management.

The DV01 is calculated as the numerical derivative of the bond price with respect to the yield to maturity (YTM). It represents the change in price per basis point change in YTM.

The formula to calculate the DV01 for a bond is:

$$
\text{DV01} = \frac{{P - P'}}{{0.0001}}
$$

where:
- $P$ is the current bond price.
- $P'$ is the bond price for a slightly higher YTM (YTM + 0.0001).

The function `calculate_dv01` uses this formula to calculate the DV01 for a given bond.

### Calculating Credit Spread:

The Credit Spread represents the additional yield above the risk-free rate that investors demand for bearing the credit risk of the bond issuer. It measures the compensation for default risk.

The Credit Spread is calculated as the difference between the bond's YTM and the risk-free interest rate.

The formula to calculate the Credit Spread for a bond is:

$$
\text{Credit Spread} = \text{YTM} - \text{Risk-Free Rate}
$$

where:
- `YTM` is the yield to maturity of the bond.
- `Risk-Free Rate` is the risk-free interest rate used as a benchmark.

The function `calculate_credit_spread`

 uses this formula to calculate the Credit Spread for a given bond.

---

I hope this documentation helps clarify the usage and functionality of the provided code. Let me know if you have any further questions!

In [1]:
def calculate_ytm(price, coupon, face_value, time_to_maturity, frequency):
    guess = 0.05  # Initial guess for YTM
    ytm = guess
    epsilon = 0.0001  # Precision of YTM calculation
    max_iterations = 100  # Maximum number of iterations for convergence

    for _ in range(max_iterations):
        present_value = 0.0
        derivative = 0.0

        for t in range(1, time_to_maturity * frequency + 1):
            present_value += coupon / frequency / (1 + ytm / frequency) ** t
            derivative -= t * coupon / frequency / (1 + ytm / frequency) ** (t + 1)

        present_value += face_value / (1 + ytm / frequency) ** (time_to_maturity * frequency)
        derivative -= time_to_maturity * face_value / frequency / (1 + ytm / frequency) ** (time_to_maturity * frequency + 1)

        error = price - present_value

        if abs(error) < epsilon:
            return ytm

        ytm += error / derivative

    raise ValueError("YTM calculation did not converge")


def calculate_dv01(price, coupon, face_value, time_to_maturity, frequency):
    ytm = calculate_ytm(price, coupon, face_value, time_to_maturity, frequency)
    price_change = price - calculate_price(coupon, face_value, time_to_maturity, frequency, ytm + 0.0001)
    dv01 = price_change / 0.0001
    return dv01


def calculate_price(coupon, face_value, time_to_maturity, frequency, ytm):
    present_value = 0.0

    for t in range(1, time_to_maturity * frequency + 1):
        present_value += coupon / frequency / (1 + ytm / frequency) ** t

    present_value += face_value / (1 + ytm / frequency) ** (time_to_maturity * frequency)
    return present_value


def calculate_credit_spread(price, coupon, face_value, time_to_maturity, frequency, risk_free_rate):
    ytm = calculate_ytm(price, coupon, face_value, time_to_maturity, frequency)
    credit_spread = ytm - risk_free_rate
    return credit_spread


# Example usage
bond_price = 980.0
coupon_payment = 40.0
face_value = 1000.0
time_to_maturity = 5
coupon_frequency = 2
risk_free_rate = 0.02

ytm = calculate_ytm(bond_price, coupon_payment, face_value, time_to_maturity, coupon_frequency)
dv01 = calculate_dv01(bond_price, coupon_payment, face_value, time_to_maturity, coupon_frequency)
credit_spread = calculate_credit_spread(bond_price, coupon_payment, face_value, time_to_maturity,
                                        coupon_frequency, risk_free_rate)

print("Yield to Maturity (YTM): {:.2%}".format(ytm))
print("DV01: {:.2f}".format(dv01))
print("Credit Spread: {:.2%}".format(credit_spread))


Yield to Maturity (YTM): 4.45%
DV01: 4384.65
Credit Spread: 2.45%
